# Olympic Project - road to gold

## Questions:
 1. How has the performance of athletes changed based on gender, and has this led to a reduction in the performance gap?
 2. Can past Olympic results reliably predict future outcomes?

## Hypothesis:


# 2. past to future

In [1]:
import pandas as pd

In [16]:
olympic_athlete_bio=pd.read_csv('Olympic_Athlete_Bio.csv',index_col=False)
olympic_athlete_event_results=pd.read_csv('Olympic_Athlete_Event_Results.csv',index_col=False)
olympic_games_medal_tally=pd.read_csv('Olympic_Games_Medal_Tally.csv',index_col=False)
olympic_results=pd.read_csv('Olympic_Results.csv',index_col=False)
olympic_country=pd.read_csv('Olympics_Country.csv',index_col=False)
olympic_games=pd.read_csv('Olympics_Games.csv',index_col=False)

In [49]:
mergedataset=pd.merge(olympic_athlete_event_results,olympic_athlete_bio.drop(columns=['name','country_noc']),on='athlete_id',how='left')
mergedataset=pd.merge(mergedataset,olympic_results.drop(columns=['event_title','edition','edition_id','sport']),on='result_id',how='left')
mergedataset=pd.merge(mergedataset,olympic_games.drop(columns=['country_noc','country_flag_url','edition_url','edition']),on='edition_id',how='left')
mergedataset=pd.merge(mergedataset,olympic_games_medal_tally.drop(columns=['edition','country','country_noc','edition']),on='edition_id',how='left')

1. The mergin is not hard. see code above
2. we need to decide which data we need and what data are we going to use.
3. depending on the data that we want to use to merge can be done differently
4. for medal count the merge is posible but we need to be careful during the analysis to not doble count
5. olympiics_country may not be neccesary